In [ ]:
!pip install lightfm

     |████████████████████████████████| 317kB 7.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705330 sha256=bc46aff43df6b9cca4103b2989f9dc3dc0e633cce02fab3572039a88cb73934e
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [ ]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/wm-project/"
%cd /content/drive/My\ Drive/wm-project/

Mounted at /content/drive
02062021MusicRecommenderTeam3.ipynb
26052021MusicRecommenderTeam3.ipynb
all_predictions_scores.csv
ALS_Hyperparameter_tuning.ipynb
als_predictions_scores1.csv
als_predictions_scores2.csv
als_predictions_scores.csv
back22052021MusicRecommenderTeam3.ipynb
df_artist.csv
df_general.csv
df_outlierDtected.csv
df_preprocessed.csv
df_test.csv
df_train_cleaned.csv
df_train_to_explore.csv
lookup_artist_names.csv
MusicRecommender_01_Preprocessing_backup.ipynb
MusicRecommender_01_Preprocessing.ipynb
MusicRecommender_02_Modelling.ipynb
nrartist_df.csv
rec_train.csv
tasks.gsheet
userid-profile.tsv
userid-timestamp-artid-artname-traid-traname.tsv
WM_Slides_08062021.pptx
/content/drive/My Drive/wm-project


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import hashlib
import pytz
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split,cross_validate,GridSearchCV
from sklearn.metrics import ndcg_score
from scipy.sparse import csr_matrix


In [ ]:
# Create Profile Dataframe

df_profile = pd.read_csv(
    'userid-profile.tsv',
    sep='\t',
    header=0,
    names=['user_id', 'gender', 'age', 'country', 'registered']
)
#convert register to DateTime in a new column
df_profile['registered_asDate'] = pd.to_datetime(df_profile['registered'],infer_datetime_format=True)
df_profile = df_profile.drop(columns='registered')
df_profile.head(3)

,user_id,gender,age,country,registered_asDate
0,user_000001,m,NaN,Japan,2006-08-13
1,user_000002,f,NaN,Peru,2006-02-24
2,user_000003,m,22.0,United States,2005-10-30


In [ ]:
trainset = pd.read_csv('df_train_cleaned.csv')


In [ ]:
trainset = trainset.drop_duplicates() # 992 users


In [ ]:
trainset = trainset.groupby('artist_id').filter(lambda x: len(x) >1)
tuning_train, tuning_test = train_test_split(trainset,
                                     test_size=0.25,
                                     random_state=41,
                                     stratify=trainset[['artist_id']])

In [ ]:
df_artist = tuning_train.drop(columns=['absolute','user_total'])
df_tunung_test_interaction = tuning_test.drop(columns=['absolute', 'user_total'])

In [ ]:
user_to_id = dict(zip(df_profile['user_id'], range(len(df_profile['user_id']))))

In [ ]:
for i,row in df_profile.iterrows():
  #df_profile.at[i, 'userID']=user_to_id[df_profile.at[i, 'user_id']]
  df_profile.loc[i,'userID']=user_to_id[df_profile.loc[i, 'user_id']]

In [ ]:
df_artist['userID']=df_artist['user_id'].map(user_to_id)
df_tunung_test_interaction['userID']=df_tunung_test_interaction['user_id'].map(user_to_id)

In [ ]:
 df_artist['userID']=df_artist['userID'].astype(np.int32)
 df_profile['userID']=df_profile['userID'].astype(np.int32)
 df_tunung_test_interaction['userID']=df_tunung_test_interaction['userID'].astype(np.int32)

In [ ]:
user_intersection_set = set.intersection(set(df_artist.userID), set(df_tunung_test_interaction.userID)) 
artist_intersection_set = set.intersection(set(df_artist.artist_id), set(df_tunung_test_interaction.artist_id)) 

In [ ]:
len(artist_intersection_set)

8117

In [ ]:
df_artist = df_artist[(df_artist.userID.isin(user_intersection_set) & df_artist.artist_id.isin(artist_intersection_set))]

In [ ]:
df_tunung_test_interaction = df_tunung_test_interaction[(df_tunung_test_interaction.userID.isin(user_intersection_set) & df_tunung_test_interaction.artist_id.isin(artist_intersection_set))]

In [ ]:
df_profile = df_profile[df_profile.userID.isin(user_intersection_set)]


In [ ]:

# dummify categorical features
user_profile_transformed = pd.get_dummies(df_profile[['userID','gender','country']], columns = ['gender','country'])

#user_profile_transformed = user_profile_transformed.sort_values('userID').reset_index().drop('index', axis=1)
user_profile_transformed.head(5)

,userID,gender_f,gender_m,country_Algeria,country_Antarctica,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Belarus,country_Belgium,country_Bosnia and Herzegovina,country_Brazil,country_British Indian Ocean Territory,country_Bulgaria,country_Canada,country_Chile,country_China,country_Colombia,"country_Congo, the Democratic Republic of the",country_Cote D'Ivoire,country_Croatia,country_Czech Republic,country_Estonia,country_Finland,country_France,country_Germany,country_Greece,country_Hungary,country_Iceland,country_India,country_Ireland,country_Israel,country_Italy,country_Japan,"country_Korea, Democratic People's Republic of",country_Latvia,country_Lithuania,country_Macedonia,country_Malta,country_Mexico,country_Morocco,country_Netherlands,country_Netherlands Antilles,country_New Zealand,country_Nicaragua,country_Northern Mariana Islands,country_Norway,country_Peru,country_Poland,country_Portugal,country_Romania,country_Russian Federation,country_Serbia,country_Singapore,country_Slovakia,country_Slovenia,country_Spain,country_Sweden,country_Switzerland,country_Thailand,country_Trinidad and Tobago,country_Tunisia,country_Turkey,country_United Kingdom,country_United States,country_United States Minor Outlying Islands,country_Venezuela,country_Zimbabwe
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
user_metadata_csr = csr_matrix(user_profile_transformed.drop('userID', axis=1).values)
user_metadata_csr

<988x68 sparse matrix of type '<class 'numpy.uint8'>'
	with 1785 stored elements in Compressed Sparse Row format>

In [ ]:
#features=np.identity(df_artist['artist_id'].unique().shape[0])

In [ ]:
#n = df_artist['artist_id'].unique().shape[0]
#features = np.ones((n, n))

In [ ]:
#features_crs=csr_matrix(features)

In [ ]:
train_interaction = pd.pivot_table(df_artist, index='userID', columns='artist_id', values='relative')
tunung_test_interaction = pd.pivot_table(df_tunung_test_interaction, index='userID', columns='artist_id', values='relative')

In [ ]:
train_interaction = train_interaction.fillna(0)
tunung_test_interaction = tunung_test_interaction.fillna(0)



In [ ]:
tuning_train_interaction_csr = csr_matrix(train_interaction.values)
tunung_test_interaction_csr = csr_matrix(tunung_test_interaction)

In [ ]:
tuning_train_interaction_csr.T.shape[0]

8117

In [ ]:
import itertools

import numpy as np

from lightfm import LightFM
from lightfm.evaluation import auc_score


def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "random_state" : 2021,
            "no_components": np.random.randint(40, 60),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "num_epochs": np.random.randint(60, 100)
        }


def random_search(train, test, num_samples=100, num_threads=1):
    """
    Sample random hyperparameters, fit a LightFM model, and evaluate it
    on the test set.

    Parameters
    ----------

    train: np.float32 coo_matrix of shape [n_users, n_items]
        Training data.
    test: np.float32 coo_matrix of shape [n_users, n_items]
        Test data.
    num_samples: int, optional
        Number of hyperparameter choices to evaluate.


    Returns
    -------

    generator of (auc_score, hyperparameter dict, fitted model)

    """

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")


        model = LightFM(**hyperparams)
        model.fit(tuning_train_interaction_csr, epochs=num_epochs, num_threads=num_threads)

        score = auc_score(model, tunung_test_interaction_csr, tuning_train_interaction_csr, num_threads=num_threads).mean()

        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)


if __name__ == "__main__":

    (score, hyperparams, model) = max(random_search(tuning_train_interaction_csr, tunung_test_interaction_csr, num_threads=1), key=lambda x: x[0])

    print("Best score {} at {}".format(score, hyperparams))


Best score 0.9074671268463135 at {'random_state': 2021, 'no_components': 40, 'learning_schedule': 'adagrad', 'loss': 'warp', 'learning_rate': 0.01846728530000181, 'num_epochs': 84}
